In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/titanic/train_and_test2.csv')

In [ ]:
df.head()

In [ ]:
df.columns.nunique()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.Embarked.value_counts()

In [ ]:
df.Embarked.describe()

In [ ]:
fig=df.Embarked.hist(bins=50)
fig.set_xlabel('2urvived')
fig.set_ylabel('Embarked')
fig.set_title('Embark')

In [ ]:
df['Embarked']=df.Embarked.fillna(2)

In [ ]:
df.isnull().sum()

# **finding an outliers**

In [ ]:
df.Fare.describe()

In [ ]:
df.describe()

In [ ]:
fig=df.boxplot(column='Fare')

In [ ]:
iqr=df['Fare'].quantile(0.75)-df['Fare'].quantile(0.25)
lower=df['Fare'].quantile(0.25)-(iqr*1.5)
upper=df['Fare'].quantile(0.75)+(iqr*1.5)
print(upper)
print(lower)

In [ ]:
df1=df.copy()
df1.loc[df1['Fare'] >=66,'Fare']=66

In [ ]:
df1.Fare.describe()

In [ ]:
from sklearn.linear_model import LogisticRegression
x=df1.drop('2urvived',axis=1)
y=df1['2urvived']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=2)


In [ ]:
lr=LogisticRegression()
lr.fit(x_train,y_train)

In [ ]:
y_pred=lr.predict(x_test)
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred)

**hyper parameter tuning**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()

In [ ]:
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
metrics.accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(start=10,stop=100,num=10)]
criterion=["gini", "entropy"]
max_features=["auto", "sqrt", "log2"]
max_depth=[int(x) for x in np.linspace(10,100,10)]
min_samples_split=[1,2,5,7,9]
min_samples_leaf=[2,4,6,8]
random_grid={'n_estimators':n_estimators,
            'criterion':criterion,
            'max_features':max_features,
            'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf}

In [ ]:
cv=RandomizedSearchCV(estimator=model,param_distributions=random_grid)
cv.fit(x_train,y_train)

In [ ]:
cv.best_params_

In [ ]:
best=cv.best_estimator_
y_pred=best.predict(x_test)
metrics.accuracy_score(y_test,y_pred)